#### Paphos Weather

This page shows various metrics for the weather in Paphos, Cyprus.

Weather data is purchased from Openweather Marketplace [here](https://home.openweathermap.org/marketplace/my_orders)



In [2]:

from datetime import date
from datetime import datetime
import time
import pandas as pd


def toDate(epoch_time):
    return datetime.fromtimestamp(epoch_time)
  


In [3]:
import pandas as pd

df=pd.read_json('/Users/walkerrowe/Documents/weather/pathosWeather.json')

In [4]:
df.columns

Index(['city_name', 'lat', 'lon', 'main', 'wind', 'clouds', 'weather', 'dt',
       'dt_iso', 'timezone', 'rain'],
      dtype='object')

In [5]:
df['dt'].map(toDate)

0        2000-01-01 02:00:00
1        2000-01-01 03:00:00
2        2000-01-01 04:00:00
3        2000-01-01 05:00:00
4        2000-01-01 06:00:00
                 ...        
181459   2020-09-12 22:00:00
181460   2020-09-12 23:00:00
181461   2020-09-13 00:00:00
181462   2020-09-13 01:00:00
181463   2020-09-13 02:00:00
Name: dt, Length: 181464, dtype: datetime64[ns]

We have to convert the epoch time to ISO Date, which is something we can read.

The map function runs a function over a series.  A series means a dataframe with one column.  When we write df['new column'] = df['some column'].map(somefunction) it adds the new columns to the dataframe.


This determines it is rained.  Need to know when it was cloudy.

In [6]:
df['dateTime']=df['dt'].map(lambda l: toDate(l).strftime("%Y-%m-%d %H:%M:%S"))

In [7]:
rainy = df[~df['rain'].isnull()]

In [8]:
rainy['rain']

46        {'1h': 0.30000000000000004}
47        {'1h': 0.30000000000000004}
48                        {'1h': 0.5}
49                        {'1h': 0.5}
50                        {'1h': 0.5}
                     ...             
178086    {'1h': 0.30000000000000004}
178091                   {'1h': 0.33}
178273                   {'1h': 0.33}
178326    {'1h': 0.30000000000000004}
178777                   {'1h': 0.33}
Name: rain, Length: 5395, dtype: object

In [9]:
df['clouds']

0          {'all': 1}
1         {'all': 15}
2         {'all': 30}
3          {'all': 1}
4         {'all': 14}
             ...     
181459    {'all': 40}
181460    {'all': 40}
181461    {'all': 40}
181462    {'all': 40}
181463    {'all': 20}
Name: clouds, Length: 181464, dtype: object

In [10]:
df['weather']

0         [{'id': 800, 'main': 'Clear', 'description': '...
1         [{'id': 801, 'main': 'Clouds', 'description': ...
2         [{'id': 802, 'main': 'Clouds', 'description': ...
3         [{'id': 800, 'main': 'Clear', 'description': '...
4         [{'id': 801, 'main': 'Clouds', 'description': ...
                                ...                        
181459    [{'id': 802, 'main': 'Clouds', 'description': ...
181460    [{'id': 802, 'main': 'Clouds', 'description': ...
181461    [{'id': 802, 'main': 'Clouds', 'description': ...
181462    [{'id': 802, 'main': 'Clouds', 'description': ...
181463    [{'id': 801, 'main': 'Clouds', 'description': ...
Name: weather, Length: 181464, dtype: object

In [11]:
rainy['isoDate']=df['dt'].map(toDate)

/var/folders/bk/np4nlklx4vs5wr6lym8k3w8w0000gn/T/ipykernel_1690/2772106319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rainy['isoDate']=df['dt'].map(toDate)


In [12]:
df.columns

Index(['city_name', 'lat', 'lon', 'main', 'wind', 'clouds', 'weather', 'dt',
       'dt_iso', 'timezone', 'rain', 'dateTime'],
      dtype='object')

We want to caculate the average temperature by date usign the mean() function.  But the problem is that the date is an object inside the **main** object.  See below how to fix that.

In [13]:
df['weatherMain']=df['weather'].map(lambda x: x[0]['main'])

In [14]:
df['weatherDesc']=df['weather'].map(lambda x: x[0]['description'])

In [15]:
df['weatherMain']

0          Clear
1         Clouds
2         Clouds
3          Clear
4         Clouds
           ...  
181459    Clouds
181460    Clouds
181461    Clouds
181462    Clouds
181463    Clouds
Name: weatherMain, Length: 181464, dtype: object

In [16]:
df['weatherDesc']

0             sky is clear
1               few clouds
2         scattered clouds
3             sky is clear
4               few clouds
                ...       
181459    scattered clouds
181460    scattered clouds
181461    scattered clouds
181462    scattered clouds
181463          few clouds
Name: weatherDesc, Length: 181464, dtype: object

In [17]:
df['temp']=df["main"].map(lambda x: x["temp"])
df['temp_min']=df["main"].map(lambda x: x["temp_min"])
df['temp_max']=df["main"].map(lambda x: x["temp_max"])

In [18]:
import numpy as np

year=[]
month=[]
row=len(df.index)
for j in range(0,row):
    date=df["dt_iso"][j]
    onlyDate=date[:len(date)-19]
    year.append(onlyDate[0:4])
    month.append(int(onlyDate[5:7]))
    
    
month=np.asarray(month)
year=np.asarray(year)

In [19]:
df['month']=pd.DataFrame(month)
df['year']=pd.DataFrame(year).astype('int32')

In [20]:
winter=df.loc[df['month'] < 4 ].groupby(['month']) 

In [21]:
december=df.loc[df['month'] == 12 ].groupby(['year','month','weatherMain'])

In [22]:
december.groups.keys()

dict_keys([(2000, 12, 'Clear'), (2000, 12, 'Clouds'), (2000, 12, 'Rain'), (2000, 12, 'Thunderstorm'), (2001, 12, 'Clear'), (2001, 12, 'Clouds'), (2001, 12, 'Rain'), (2001, 12, 'Thunderstorm'), (2002, 12, 'Clear'), (2002, 12, 'Clouds'), (2002, 12, 'Rain'), (2002, 12, 'Thunderstorm'), (2003, 12, 'Clear'), (2003, 12, 'Clouds'), (2003, 12, 'Rain'), (2003, 12, 'Thunderstorm'), (2004, 12, 'Clear'), (2004, 12, 'Clouds'), (2004, 12, 'Rain'), (2004, 12, 'Thunderstorm'), (2005, 12, 'Clear'), (2005, 12, 'Clouds'), (2005, 12, 'Haze'), (2005, 12, 'Mist'), (2005, 12, 'Rain'), (2006, 12, 'Clear'), (2006, 12, 'Clouds'), (2006, 12, 'Rain'), (2006, 12, 'Thunderstorm'), (2007, 12, 'Clear'), (2007, 12, 'Clouds'), (2007, 12, 'Rain'), (2007, 12, 'Thunderstorm'), (2008, 12, 'Clear'), (2008, 12, 'Clouds'), (2008, 12, 'Rain'), (2008, 12, 'Thunderstorm'), (2009, 12, 'Clear'), (2009, 12, 'Clouds'), (2009, 12, 'Haze'), (2009, 12, 'Rain'), (2009, 12, 'Thunderstorm'), (2010, 12, 'Clear'), (2010, 12, 'Clouds'), (201

In [23]:
df['year']

0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
181459    2020
181460    2020
181461    2020
181462    2020
181463    2020
Name: year, Length: 181464, dtype: int32

In [28]:
this=df.groupby(['month','weatherMain'])['dt'].count()
pd.set_option('display.max_rows', this.shape[0]+1)
print(this)

month  weatherMain 
1      Clear            1309
       Clouds          12687
       Dust              123
       Fog                 1
       Haze               26
       Mist               18
       Rain             1105
       Squall              3
       Thunderstorm      352
2      Clear            1776
       Clouds          11258
       Dust              158
       Fog                 1
       Haze               62
       Mist               43
       Rain              702
       Squall              4
       Thunderstorm      252
3      Clear            2746
       Clouds          11907
       Dust              284
       Fog                 8
       Haze               67
       Mist               53
       Rain              428
       Thunderstorm      131
4      Clear            3369
       Clouds          10958
       Dust              158
       Fog                14
       Haze              111
       Mist              129
       Rain              296
       Squall          